# RESEARCH NOTEBOOK --> SUPERTREND

In [ ]:
import pandas as pd
import pandas_ta as ta  # noqa: F401

candles = pd.read_csv(
    "/Users/dardonacci/Documents/work/dashboard/data/candles/candles_binance_perpetual_WLD-USDT_3m.csv")

In [ ]:
candles.head()

In [ ]:
super_trend_lenght = 20
super_trend_multiplier = 3
candles.ta.supertrend(length=super_trend_lenght, multiplier=super_trend_multiplier)

In [ ]:
candles.ta.supertrend(length=super_trend_lenght, multiplier=super_trend_multiplier, append=True)

In [ ]:
candles["date"] = pd.to_datetime(candles["timestamp"], unit='ms')
candles.head()

In [ ]:
import plotly.graph_objects as go

# We are going to use just a subset to see the graph better
candles = candles.tail(800)

# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=candles['date'],
    open=candles['open'],
    high=candles['high'],
    low=candles['low'],
    close=candles['close'])
])
fig.show()

In [ ]:
super_trend_long = candles[candles["SUPERTd_20_3.0"] == 1]
super_trend_short = candles[candles["SUPERTd_20_3.0"] == -1]
# Add the SuperTrend line
fig.add_trace(go.Scatter(x=super_trend_long['date'], y=super_trend_long['SUPERT_20_3.0'],
                         mode='markers',
                         name='SuperTrend Long',
                         line=dict(color="green"),
                         ))
# Add the SuperTrend line
fig.add_trace(go.Scatter(x=super_trend_short['date'], y=super_trend_short['SUPERT_20_3.0'],
                         mode='markers',
                         name='SuperTrend Short',
                         line=dict(color="red")))

In [ ]:
percentage_threshold = 0.01  # This is an example threshold value

candles["percentage_distance"] = abs(candles["close"] - candles["SUPERT_20_3.0"]) / candles["close"]

candles["signal"] = 0
long_condition = (candles["SUPERTd_20_3.0"] == 1) & (candles["percentage_distance"] < percentage_threshold)
short_condition = (candles["SUPERTd_20_3.0"] == -1) & (candles["percentage_distance"] < percentage_threshold)

candles.loc[long_condition, "signal"] = 1
candles.loc[short_condition, "signal"] = -1

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, subplot_titles=('OHLC', 'Signal'),
                    row_heights=[0.7, 0.3])

# Add candlestick
fig.add_trace(go.Candlestick(
    x=candles['date'],
    open=candles['open'],
    high=candles['high'],
    low=candles['low'],
    close=candles['close']),
    row=1, col=1)

# Add the SuperTrend line
fig.add_trace(go.Scatter(x=super_trend_long['date'], y=super_trend_long['SUPERT_20_3.0'],
                         mode='markers',
                         name='SuperTrend Long',
                         line=dict(color="green")),
              row=1, col=1)
# Add the SuperTrend line
fig.add_trace(go.Scatter(x=super_trend_short['date'], y=super_trend_short['SUPERT_20_3.0'],
                         mode='markers',
                         name='SuperTrend Short',
                         line=dict(color="red")),
              row=1, col=1)

# Add the signal line
fig.add_trace(go.Scatter(x=candles['date'], y=candles['signal'],
                         mode='lines',
                         name='SuperTrend',
                         line=dict(color="white")),
              row=2, col=1)

# Update x-axis and grid properties
fig.update_xaxes(showline=True, linewidth=2, linecolor='grey', gridcolor='lightgrey')
fig.update_yaxes(showline=True, linewidth=2, linecolor='grey', gridcolor='lightgrey')

# Update layout to adjust the size and title
fig.update_layout(height=800, title_text="OHLC Chart with SuperTrend and Signals",
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)
